In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
Image.open("/kaggle/input/dacon-bird/train/TRAIN_11814.jpg")

In [ ]:
Image.open("/kaggle/input/dacon-bird/upscale_train/TRAIN_00248.png")

In [ ]:
#정답 클래스가 어떻게 되어있는지 확인하어야한다 대회진행할때
train=pd.read_csv("/kaggle/input/dacon-bird/train.csv")
train

In [ ]:
train["label"].value_counts()

In [ ]:
pd.options.display.max_colwidth=200
train["img_path"]="/kaggle/input/dacon-bird"+train["img_path"].str[1:]
train

In [ ]:
test=pd.read_csv("/kaggle/input/dacon-bird/test.csv")


In [ ]:
pd.options.display.max_colwidth=200
test["img_path"]="/kaggle/input/dacon-bird"+test["img_path"].str[1:]
test

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the KFold cross-validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

idg = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


#idg 에 train 에 만 넣어도
idg2=ImageDataGenerator()


#train_generator=idg.flow_from_dataframe(x_train,x_col="img_path",y_col="label")
#valid_generator=idg.flow_from_dataframe(x_valid,x_col="img_path",y_col="label")


In [ ]:
# Track performance for each fold
fold_no = 1
result=0
for train_idx, valid_idx in kfold.split(train):
    # Split the data
    x_train, x_valid = train.iloc[train_idx], train.iloc[valid_idx]
    
    # Create generators
    train_generator = idg.flow_from_dataframe(x_train, x_col='img_path', y_col='label')
    valid_generator = idg2.flow_from_dataframe(x_valid, x_col='img_path', y_col='label')
    
    # Define the model architecture
    model = Sequential([
        EfficientNetB1(include_top=False, pooling='avg', weights='imagenet'),
        Dense(25, activation='softmax')
    ])
    model.compile(metrics=['acc'], optimizer='adam', loss='categorical_crossentropy')
    
    # EarlyStopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    rl=ReduceLROnPlateau(patience=2)
    
    # Fit the model
    print(f'Training for fold {fold_no}...')
    model.fit(
        train_generator,
        epochs=20,
        validation_data=valid_generator,
        callbacks=[early_stopping,rl]
    )
    
    test_generator=idg2.flow_from_dataframe(test, x_col='img_path', y_col=None, shuffle=False, class_mode=None )
    result+=model.predict(test_generator)/5


In [ ]:
sub=pd.read_csv("/kaggle/input/dacon-bird/sample_submission.csv")

In [ ]:
sub["label"]=result.argmax(1)

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(train['label'])
sub['label']=le.inverse_transform(sub['label'])


In [ ]:
sub

In [ ]:
sub.to_csv("sub123123.csv",index=0)